### Every-visit Monte Carlo with Exploring Starts for estimating an optimal Blackjack policy

This corresponds to the algorithm described in Chapter 5.3 of [_Reinforcement Learning: An Introduction_](http://incompleteideas.net/book/the-book-2nd.html), by Sutton and Barto.

In [69]:
# Dependencies
import gym

from rl_agents.blackjack.MonteCarloAgent import MonteCarloAgent

### Initialize Blackjack agent and environment

In [70]:
RANDOM_SEED = 2

env = gym.make('Blackjack-v0')
env.seed(seed=RANDOM_SEED)
agent = MonteCarloAgent(action_space=env.action_space,
                        obs_space=env.observation_space,
                        seed=RANDOM_SEED)

TypeError: __init__() got an unexpected keyword argument 'seed'

AttributeError: 'MonteCarloAgent' object has no attribute 'actions'

### Run Experiment

In [65]:
 for i_episode in range(5):
    observation = env.reset()
    reward = 0
    done = False
    t = 0
    episode_ts = []
    while not done:
        action = agent.agent_step(reward=reward,
                                  observation=observation)

        observation, reward, done, info = env.step(action)

        episode_ts.append((action, observation, reward))

        if done:
            print("Episode finished after {} timesteps".format(t + 1))
            break

        t += 1

    agent.agent_end(episode_ts=episode_ts)

AttributeError: 'MonteCarloAgent' object has no attribute 'select_action'

### Analyze agent's policy